In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import time


In [2]:
mp_pose = mp.solutions.pose

In [3]:
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

In [4]:
cap = cv2.VideoCapture(0)  # or use your image/video

all_data = []

In [5]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Confidence calculation
        confidences = [landmark.visibility for landmark in results.pose_landmarks.landmark]
        avg_confidence = sum(confidences) / len(confidences)

         # Calculate accuracy in percentage
        accuracy_percentage = avg_confidence * 100  # Multiply by 100 to get percentage

        # Save timestamp and accuracy
        data = {
            'timestamp': time.time(),
            'avg_confidence': avg_confidence,
            'accuracy_percentage': accuracy_percentage
        }
        all_data.append(data)
    
        # Display Confidence and Accuracy on the frame
        cv2.putText(frame, f'Confidence: {avg_confidence:.2f}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, f'Accuracy: {accuracy_percentage:.2f}%', (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Pose Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
# Save results to CSV
df = pd.DataFrame(all_data)
df.to_csv("pose_accuracy.csv", index=False)
print("✅ Accuracy data saved to pose_accuracy.csv")


✅ Accuracy data saved to pose_accuracy.csv


In [7]:
df

,timestamp,avg_confidence,accuracy_percentage
0,1.744527e+09,0.541796,54.179559
1,1.744527e+09,0.541272,54.127185
2,1.744527e+09,0.540865,54.086485
3,1.744527e+09,0.536526,53.652633
4,1.744527e+09,0.536336,53.633561
...,...,...,...
73,1.744527e+09,0.421707,42.170731
74,1.744527e+09,0.420815,42.081497
75,1.744527e+09,0.420984,42.098357
76,1.744527e+09,0.420949,42.094864


In [8]:
import sqlite3

In [9]:
# Step 1: Connect to DB
conn = sqlite3.connect('results.db')
cursor = conn.cursor()

In [10]:
# Drop table if it exists
cursor.execute("DROP TABLE IF EXISTS results")
# Step 2: Create Table (if not exists)


# Create it with the correct structure
cursor.execute('''
    CREATE TABLE results (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        model_name TEXT,
        avg_confidence REAL,
        accuracy REAL
    )
''')

conn.commit()


In [11]:
#cursor.execute("INSERT INTO Pose_detection (model_name, accuracy) VALUES (?, ?)", ("Pose Detection", accuracy_percentage))
#conn.commit()
#avg_confidence = df["avg_confidence"].mean()

for accuracy in df["accuracy_percentage"]:
    cursor.execute(
        "INSERT INTO results (model_name, avg_confidence, accuracy) VALUES (?, ?, ?)",
        ("Pose Detection", avg_confidence, accuracy)
    )

conn.commit()



In [12]:
# Step 4: Verify
cursor.execute("SELECT * FROM results")
for row in cursor.fetchall():
    print(row)

(1, 'Pose Detection', 0.4215748548389952, 54.179559236669746)
(2, 'Pose Detection', 0.4215748548389952, 54.12718470481306)
(3, 'Pose Detection', 0.4215748548389952, 54.08648537777481)
(4, 'Pose Detection', 0.4215748548389952, 53.65263317818806)
(5, 'Pose Detection', 0.4215748548389952, 53.63356135970786)
(6, 'Pose Detection', 0.4215748548389952, 53.5991848568246)
(7, 'Pose Detection', 0.4215748548389952, 53.41323228578514)
(8, 'Pose Detection', 0.4215748548389952, 53.194305499456476)
(9, 'Pose Detection', 0.4215748548389952, 53.31876533285443)
(10, 'Pose Detection', 0.4215748548389952, 53.413731087582136)
(11, 'Pose Detection', 0.4215748548389952, 53.25594791813108)
(12, 'Pose Detection', 0.4215748548389952, 53.08755326083029)
(13, 'Pose Detection', 0.4215748548389952, 53.16175373485418)
(14, 'Pose Detection', 0.4215748548389952, 53.38011467747768)
(15, 'Pose Detection', 0.4215748548389952, 53.46445252492258)
(16, 'Pose Detection', 0.4215748548389952, 53.34543751430994)
(17, 'Pose Dete

In [13]:
conn.close()